# 写在前面

2022/06/12

整理自labuladong


数据结构的基本存储方式就是链式和顺序两种，基本操作就是增删查改，遍历方式无非迭代和递归。

算法的本质就是“穷举”。

穷举的难点：
- 无遗漏，
- 无冗余：聪明地请举，消耗尽可能少的资源求出答案。

# 数组/链表

- 遍历

- 双指针
    - 滑动窗口
- 前缀和
- 差分

## 遍历

只要递归形式的遍历，都可以有前序位置和后序位置。

In [ ]:
# 迭代遍历数组

def traverse(nums):
    for num in nums:
        ...
    return ...

# 递归遍历数组

def traverse(nums, i):
    if i == len(nums):
        ...
        return
    
    # 前序位置操作
    ...
    traverse(nums, i + 1)
    
    # 后序位置操作
    ...
    return ...

# 迭代遍历链表
def traverse(head):
    p = head
    while p:
        # ...
        p = p.next
        
    return ...

# 递归遍历链表
def traverse(head):
    if head is None:
        return ...
    
    # 前序位置操作
    ...
    traverse(head.next)
    
    # 后序位置操作
    ...
    return ...
    


## 双指针

### 滑动窗口

关键点：
- 加入新item时，应该如何更新窗口
- 什么时候要缩小窗口？
- 缩小窗口时，应该更新什么值？

采取的是左开右闭的区间，好处是：当l=r=0的时候，窗口内没有值，对应于初始情况。

In [ ]:
# 滑动窗口模板

def slidingWindow(s, t):
    l = 0
    r = 0
    res = ...
    while r < len(s):
        item = s[r]
        r += 1
        # update the window
        # ....
        
        print("Debug: window[{}: {}]".format(l, r))
        
        while needsShrink(window):
            removed = s[l]
            l += 1
            
            # update the window
            # ...
    return res...


## 前缀和技巧

适用于：原始数组不会被修改，频繁地计算某个区间的和

例题：304

模板：两个操作：初始化，和查询。

实现起来有两种，
- 一种是带冗余的，就不用考虑起始位置了，空间会多用一些。下标处理的时候，要多留意，多了一位,presum[i]代表的是前i个
- 一种是不带冗余的，节省空间，但是需要单独处理边缘

In [ ]:
# 前缀和模板

# 一维冗余版本
class PreSum(object):
    def __init__(self, nums):
        self.prefix = [0 for _ in range(len(nums)+1)] # 这里多了个1，冗余的空间，减少分类讨论

        for i in range(len(nums)):
            self.prefix[i+1] = self.prefix[i]+nums[i]
            
    def query(self, i, j):
        # by default, i <= j and it is the closed sublist
        return self.prefix[j+1] - self.prefix[i] # 用j+1计算


# 一维非冗余版本
class PreSum(object):
    def __init__(self, nums):
        self.prefix = [0 for _ in range(len(nums))]
        # 单独处理第一个元素
        self.prefix[0] = nums[0]
        
        # repeat for the others
        for i in range(1, len(nums)):
            self.prefix[i] = self.prefix[i-1]+nums[i]
            
    def query(self, i, j):
        # by default, i <= j and it is the closed sublist
        
        if i == 0: # 需要单独处理查询开始为0的情况
            return self.prefix[j]
        return self.prefix[j] - self.prefix[i-1]

# 二维冗余版本
class NumMatrix:

    def __init__(self, matrix: List[List[int]]):
        self.m = len(matrix)
        self.n = len(matrix[0])
        self.presum = [[0 for _ in range(self.n+1)] for _ in range(self.m+1)] # 多了一圈的空间
        
        for l in range(self.m):
            for c in range(self.n):
                # 注意：presum里的下标都要+1
                self.presum[l+1][c+1] = self.presum[l][c+1] + self.presum[l+1][c] - self.presum[l][c] + matrix[l][c]

    def sumRegion(self, row1: int, col1: int, row2: int, col2: int) -> int:
        # print(row1, col1, row2, col2)
        return self.presum[row2+1][col2+1] - self.presum[row1][col2+1] - self.presum[row2+1][col1] + self.presum[row1][col1]



# 二维非冗余版本
class NumMatrix:
    def __init__(self, matrix: List[List[int]]):
        self.m = len(matrix)
        self.n = len(matrix[0])
        self.presum = [[0 for _ in range(self.n)] for _ in range(self.m)]
        # initialization
        # 需要单独为第一行第一列的元素初始化
        self.presum[0][0] = matrix[0][0]
        # 需要单独为第一行的元素初始化
        for c in range(1, self.n):
            self.presum[0][c] = self.presum[0][c-1] + matrix[0][c]
        # 需要单独为第一列的元素初始化
        for l in range(1, self.m):
            self.presum[l][0] = self.presum[l-1][0] + matrix[l][0]
        # calculate for others
        for l in range(1, self.m):
            for c in range(1, self.n):
                self.presum[l][c] = self.presum[l-1][c] + self.presum[l][c-1] - self.presum[l-1][c-1] + matrix[l][c]

    def sumRegion(self, row1: int, col1: int, row2: int, col2: int) -> int:
        # 需要单独处理第一行第一列，第一行，第一列
        if row1 == 0 and col1 == 0:
            return self.presum[row2][col2]
        if row1 == 0:
            return self.presum[row2][col2] - self.presum[row2][col1-1]
        if col1 == 0:
            return self.presum[row2][col2] - self.presum[row1-1][col2]

        return self.presum[row2][col2] - self.presum[row1-1][col2] - self.presum[row2][col1-1] + self.presum[row1-1][col1 - 1]



## 差分技巧

适用于：频繁地对子区间进行增减操作

模板：只需要改动区间开头和区间结尾的后一位的值即可

三个操作：初始化、改动和查询

例题：1109，想到一个好玩的变式：假如说每个飞机都有💺数量的上限，有一个预定流，对于每一个预定，返回是否预定成功。预定成功的意思是，每一个航班所有被预定的座位数没有超过它的上限。如果预定失败了，则终止剩下的预定。

这个变式怎么解？就别用差分了吧，记录每一个飞机的座位数。为什么？因为每一次预定，需要检查预定区间里所有飞机的状态，这一步复杂度没法省，而差分数组节省地恰恰就是这步骤的复杂度。

In [ ]:
# 模板
class DiffNums(object):
    def __init__(self, nums):
        self.len = len(nums)
        self.diff = [num for _ in nums]
        for i in range(1, self.len):
            self.diff[i] = nums[i] - nums[i-1]
            
    def incremental(self, i, j, val):
        self.diff[i] += val
        if j +1 < self.len:
            self.diff[j+1] -= val
        
    def query(self):
        res = []
        if self.len > 0:
            res = [self.diff[0]]
            for i in range(1, self.len):
                res.append(res[-1] + self.diff[i])
        return res
        

## 单调栈

关键在于：弄清什么时候入栈，什么时候出栈。

In [ ]:
# 单调栈模板
def nextGreaterElement(nums):
    n = len(nums)
    res = [0 for num in nums]
    s = []
    for i in range(n-1, -1, -1):
        
        while s and s[-1] <= nums[i]:
            s.pop()
            
        res[i] = -1 if not s else s[-1]
        s.append(nums[i])
    return res

# 二叉树

二叉树的递归解法分为两类思路：
- 遍历一遍，得出答案：回溯算法。用一个`traverse`函数配合外部变量来实现。**“遍历”**的思维模式。
- 分解问题，计算出答案：动态规划/递归。通过子问题（子树）的答案推导出原问题的答案。充分利用函数的返回值，**“分解问题”**的思维模式。

无论哪一种模式，都要思考：

如果单独抽出一个二叉树节点，
- 它需要做什么事情？
- 需要在什么时候（前/中/后序位置）做？
    - 前序位置的代码：只能从函数参数中获取父节点传递来的数据
    - 后序位置的代码：不仅可以获取参数数据，还可以获取到子树通过函数返回值传递回来的数据
        - 一旦发现题目和子树有关，大概率要给函数设置合理的定义和返回值，在后序位置写代码。


二叉树的所有问题，就是让你在前中后序位置注入巧妙的代码逻辑，去达到自己的目的。只需要单独思考每一个节点应该做什么！

In [ ]:
def traverse(node):
    # base case 
    # ...
    
    # preorder operations
    # ...
    
    traverse(node.left)
    
    # inorder operations
    # ...
    
    traverse(node.right)
    
    # postorder operations
    # ...
    return ...

# 二分搜索

分析二分查找的一个技巧是：不要出现else，而是把所有情况用else if写清楚，这样可以清楚地展现所有细节

应用场景：从题目中抽象出一个自变量`x`，一个关于`x`的函数`f(x)`，以及一个目标值`target`，这三个数满足以下条件：

1. `f(x)`是`x`上的单调函数
2. 题目是让你计算满足约束条件`f(x) == target`时的`x`的值。

常见问题：在最大中求最小等，转化为可判定问题，二分搜索答案。

解题步骤：
1. 确定`x`，`f(x)`和`target`分别是什么，写出`f(x)`的代码
2. 找到`x`的取值范围，作为二分搜索的搜索区间，初始化left和right变量
3. 根据题目的要求，确定应该使用搜索左侧还是右侧的二分搜索算法，写出代码。

In [ ]:
# 二分搜索模板

def binarySearch(nums, target):
    l = 0
    r = ...
    while ...:
        m = l + (r-l)//2 # 不用 m = (l + r) // 2，防止溢出
        if nums[m] == target:
            ...
        elif nums[m] < target:
            ...
        elif nums[m] > target:
            ...
    return ...

# 搜索左边界的二分查找
def left_bound(nums, target):
    left = 0
    right = len(nums)
    
    while left < right:
        mid = left + (right - left) // 2
        if nums[mid] == target:
            right = mid
        elif nums[mid] < target:
            left = mid + 1
        else:
            # nums[mid] > target:
            right = mid
    return left
        
# 搜索左边界的二分查找，抽象版本
def left_bound(nums, target):
    left = 0
    right = len(nums)
    
    while left < right:
        mid = left + (right - left) // 2
        mid_val = f(mid, nums)
        if mid_val == target:
            right = mid
        elif mid_val < target:
            left = mid + 1
        else:
            # mid_val > target:
            right = mid
    return left

# 搜索右边界的二分查找
def right_bound(nums, target):
    left = 0
    right = len(nums)
    
    while left < right:
        mid = left + (right - left) // 2
        if nums[mid] == target:
            left = mid + 1
        elif nums[mid] < target:
            left = mid + 1
        else:
            # nums[mid] > target:
            right = mid
    return left - 1

In [ ]:
# 二分搜索解题模板

def f(x):
    ...
    
def solution(nums, target):
    left = 0
    right = len(nums)
    while left < right:
        if f(mid) == target:
            # 问自己，是求左边界还是右边界
            ...
        elif f(mid) > target:
            # 问自己，怎么让f(x)小一点？
        else:
            #  if f(mid) < target:
            # 问自己，怎么让f(x)大一点？
    return left

# 图论



# 回溯算法

回溯算法就是N叉树的前后序遍历问题。

# 动态规划

1. 先写出暴力穷举解法（状态转移方程）
2. 加个备忘录就是自顶向下的递归了
3. 改一改就是自底向上的迭代了
4. 空间复杂度的优化
